In [1]:
time_start = "2020-12-31"
time_end =  "2021-06-30"

In [2]:
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
dataframe_list = pd.DataFrame()
for f, _, i in walk("/data/liguichuan/factor_lib_ti0/labeled_data/5d/20210331"):
    for j in tqdm(i):
        dataframe_list = pd.concat([dataframe_list,pd.read_parquet(f + "/" + j)],axis=0)
dataframe_list

100%|██████████| 21/21 [00:05<00:00,  4.06it/s]


,timestamp,ticker,pv_daily_fangxin_onvmp_factor,alpha_onc_revised,pv_daily_fangxin_30min_TD_highlowclosesum_30m_zz800_5h_V_pos,pv_daily_fangxin_30min_TD_highlowclosevolume_30m_zz800_5h_V_neg,pv_daily_fangxin_barra_alp,pv_daily_currentopen_open_ratio,pv_daily_fangxin_30min_TD_preturnstd01_30m_zz800_1d4h_V_neg,pv_daily_fangxin_abs_rel_ratio_10_pos,...,pv_daily_fangxin_TB_bias04_1d_zz800_5d_V_pos,pv_daily_fangxin_TA_amp001_1d_zz800_5d_V_pos,pv_daily_fangxin_onvmp_factor_mean_5,pv_daily_currentopen_open_ratio_mean_3,pv_daily_fangxin_onc_factor_std_5,pv_daily_fangxin_abs_rel_ratio_5_pos,pv_daily_fangxin_alt_rt_close2h1,pv_daily_fangxin_c_min_ratio_20,pv_daily_fangxin_c_min_ratio_10,target
0,2017-01-03 09:30:00,000651,-0.681654,0.659641,1.237512,2.470494,-0.582581,-0.595604,1.0,1.0,...,1.0,2.661833,0.248848,-0.442206,-0.362527,1.0,-0.918876,-0.331262,0.122829,-0.338307
1,2017-01-03 09:30:00,000002,-0.660462,-0.312563,-0.180847,2.470494,-0.849203,-1.113090,1.0,1.0,...,1.0,2.661833,-0.239625,-1.102746,0.017577,1.0,-0.975593,-0.608850,-0.316651,-0.252614
2,2017-01-03 09:30:00,002065,-2.265180,0.040441,3.847892,2.470494,-3.316579,-1.811118,1.0,1.0,...,1.0,2.661833,0.732424,-1.263933,2.014061,1.0,-0.342771,-0.337805,-0.364199,-0.999581
3,2017-01-03 09:30:00,002024,2.150972,0.040441,0.223929,2.470494,-0.901429,0.542641,1.0,1.0,...,1.0,2.661833,1.063530,0.297735,1.964825,1.0,-0.965956,-0.771722,-0.574511,-0.613760
4,2017-01-03 09:30:00,000635,-3.411397,0.040441,1.938823,2.470494,-3.463908,-3.483466,1.0,1.0,...,1.0,2.661833,0.171116,-2.152819,3.168597,1.0,-3.226028,3.366683,3.095237,-0.187823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,2020-09-30 09:30:00,002410,1.060003,-2.572184,-0.755433,2.361176,1.877067,1.717936,1.0,1.0,...,1.0,-2.351677,0.863170,1.561495,0.302867,1.0,1.805092,0.746824,1.389110,1.165332
3429,2020-09-30 09:30:00,002714,1.203517,-0.290483,-0.544024,2.361176,1.757388,0.936983,1.0,1.0,...,1.0,-2.351677,1.524303,0.392107,0.543736,1.0,0.221880,0.447035,0.984824,-0.157514
3430,2020-09-30 09:30:00,000661,1.356401,1.804779,-0.171822,2.361176,0.021634,0.730565,1.0,1.0,...,1.0,-2.351677,0.997743,0.721822,-0.349514,1.0,0.903561,-0.067663,0.271973,-0.306495
3431,2020-09-30 09:30:00,600588,0.862688,0.371054,-0.496678,2.361176,1.156435,0.710229,1.0,1.0,...,1.0,-2.351677,0.475991,1.462331,0.423895,1.0,1.073898,0.241972,0.681378,1.000409


In [3]:
train_frame = dataframe_list[dataframe_list['timestamp'] < pd.to_datetime(time_start)]
test_frame = dataframe_list[(dataframe_list['timestamp'] > pd.to_datetime(time_start))
                            & (dataframe_list['timestamp'] < pd.to_datetime(time_end))]
display(train_frame)
display(test_frame)
del dataframe_list

,timestamp,ticker,pv_daily_fangxin_onvmp_factor,alpha_onc_revised,pv_daily_fangxin_30min_TD_highlowclosesum_30m_zz800_5h_V_pos,pv_daily_fangxin_30min_TD_highlowclosevolume_30m_zz800_5h_V_neg,pv_daily_fangxin_barra_alp,pv_daily_currentopen_open_ratio,pv_daily_fangxin_30min_TD_preturnstd01_30m_zz800_1d4h_V_neg,pv_daily_fangxin_abs_rel_ratio_10_pos,...,pv_daily_fangxin_TB_bias04_1d_zz800_5d_V_pos,pv_daily_fangxin_TA_amp001_1d_zz800_5d_V_pos,pv_daily_fangxin_onvmp_factor_mean_5,pv_daily_currentopen_open_ratio_mean_3,pv_daily_fangxin_onc_factor_std_5,pv_daily_fangxin_abs_rel_ratio_5_pos,pv_daily_fangxin_alt_rt_close2h1,pv_daily_fangxin_c_min_ratio_20,pv_daily_fangxin_c_min_ratio_10,target
0,2017-01-03 09:30:00,000651,-0.681654,0.659641,1.237512,2.470494,-0.582581,-0.595604,1.0,1.0,...,1.0,2.661833,0.248848,-0.442206,-0.362527,1.0,-0.918876,-0.331262,0.122829,-0.338307
1,2017-01-03 09:30:00,000002,-0.660462,-0.312563,-0.180847,2.470494,-0.849203,-1.113090,1.0,1.0,...,1.0,2.661833,-0.239625,-1.102746,0.017577,1.0,-0.975593,-0.608850,-0.316651,-0.252614
2,2017-01-03 09:30:00,002065,-2.265180,0.040441,3.847892,2.470494,-3.316579,-1.811118,1.0,1.0,...,1.0,2.661833,0.732424,-1.263933,2.014061,1.0,-0.342771,-0.337805,-0.364199,-0.999581
3,2017-01-03 09:30:00,002024,2.150972,0.040441,0.223929,2.470494,-0.901429,0.542641,1.0,1.0,...,1.0,2.661833,1.063530,0.297735,1.964825,1.0,-0.965956,-0.771722,-0.574511,-0.613760
4,2017-01-03 09:30:00,000635,-3.411397,0.040441,1.938823,2.470494,-3.463908,-3.483466,1.0,1.0,...,1.0,2.661833,0.171116,-2.152819,3.168597,1.0,-3.226028,3.366683,3.095237,-0.187823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,2020-09-30 09:30:00,002410,1.060003,-2.572184,-0.755433,2.361176,1.877067,1.717936,1.0,1.0,...,1.0,-2.351677,0.863170,1.561495,0.302867,1.0,1.805092,0.746824,1.389110,1.165332
3429,2020-09-30 09:30:00,002714,1.203517,-0.290483,-0.544024,2.361176,1.757388,0.936983,1.0,1.0,...,1.0,-2.351677,1.524303,0.392107,0.543736,1.0,0.221880,0.447035,0.984824,-0.157514
3430,2020-09-30 09:30:00,000661,1.356401,1.804779,-0.171822,2.361176,0.021634,0.730565,1.0,1.0,...,1.0,-2.351677,0.997743,0.721822,-0.349514,1.0,0.903561,-0.067663,0.271973,-0.306495
3431,2020-09-30 09:30:00,600588,0.862688,0.371054,-0.496678,2.361176,1.156435,0.710229,1.0,1.0,...,1.0,-2.351677,0.475991,1.462331,0.423895,1.0,1.073898,0.241972,0.681378,1.000409


,timestamp,ticker,pv_daily_fangxin_onvmp_factor,alpha_onc_revised,pv_daily_fangxin_30min_TD_highlowclosesum_30m_zz800_5h_V_pos,pv_daily_fangxin_30min_TD_highlowclosevolume_30m_zz800_5h_V_neg,pv_daily_fangxin_barra_alp,pv_daily_currentopen_open_ratio,pv_daily_fangxin_30min_TD_preturnstd01_30m_zz800_1d4h_V_neg,pv_daily_fangxin_abs_rel_ratio_10_pos,...,pv_daily_fangxin_TB_bias04_1d_zz800_5d_V_pos,pv_daily_fangxin_TA_amp001_1d_zz800_5d_V_pos,pv_daily_fangxin_onvmp_factor_mean_5,pv_daily_currentopen_open_ratio_mean_3,pv_daily_fangxin_onc_factor_std_5,pv_daily_fangxin_abs_rel_ratio_5_pos,pv_daily_fangxin_alt_rt_close2h1,pv_daily_fangxin_c_min_ratio_20,pv_daily_fangxin_c_min_ratio_10,target
0,2021-04-01 09:30:00,002893,-3.342067,-0.072456,-0.010892,-0.667746,3.196831,-0.681232,6.107599,1.0,...,1.0,-2.203868,-2.752647,3.687042,3.100064,1.0,-0.228537,3.360352,2.953430,-0.892853
1,2021-04-01 09:30:00,600578,-0.387978,-0.072456,-0.010892,2.355450,3.196831,-0.164340,4.224982,1.0,...,1.0,-1.677491,3.358650,3.687042,3.100064,1.0,2.011674,3.360352,2.953430,-3.204961
2,2021-04-01 09:30:00,603080,-3.342067,-0.072456,-4.175977,1.493767,3.196831,-3.503692,3.808660,1.0,...,1.0,-0.728207,3.358650,2.890623,3.100064,1.0,2.101096,3.360352,2.953430,0.308000
3,2021-04-01 09:30:00,002941,-2.629117,-0.072456,-0.010892,0.321699,3.196831,-2.043998,4.259800,1.0,...,1.0,-0.744239,3.358650,3.687042,3.100064,1.0,2.480506,3.360352,2.953430,-3.485069
4,2021-04-01 09:30:00,600339,-2.885687,-0.072456,-4.175977,2.355450,3.196831,-1.567390,5.075221,1.0,...,1.0,-0.403546,3.358650,3.687042,3.100064,1.0,-0.228537,3.360352,2.953430,-2.880500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359,2020-12-31 09:30:00,600276,0.464350,-0.013809,-0.214182,2.236800,1.193360,1.128652,1.000000,1.0,...,1.0,-2.302472,1.119608,-0.472200,-0.545299,1.0,0.311127,2.364765,2.645691,0.385904
3360,2020-12-31 09:30:00,600570,3.355659,2.686748,0.223915,2.236800,2.925218,3.185883,1.000000,1.0,...,1.0,-2.302472,1.171115,2.258461,0.596313,1.0,2.976783,1.258267,1.395389,-0.425520
3361,2020-12-31 09:30:00,600621,0.665403,-0.766664,0.653074,2.236800,2.332617,2.782492,1.000000,1.0,...,1.0,-2.302472,-1.404781,2.250870,-0.565051,1.0,2.376212,1.655261,1.287760,-0.328209
3362,2020-12-31 09:30:00,600601,0.146007,-0.471641,-1.417362,-0.532204,0.524097,0.051722,1.000000,1.0,...,1.0,-2.302472,0.262013,-0.327140,-0.919080,1.0,0.730325,-0.040375,0.134872,0.111994


In [4]:
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天

trainx , trainy = [] , []
for ticker in tqdm(train_frame['ticker'].drop_duplicates()):
    one_data = train_frame[train_frame['ticker'] == ticker]
    one_data = one_data.set_index(['timestamp','ticker'])
    array = np.array(one_data)
    for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        trainx.append(array[i:i+day,:-1].T)
        trainy.append(array[i+day-1][-1])
trainx  , trainy = np.array(trainx) , np.array(trainy).reshape(-1,1) # x = (153, 9, 30) , y = (153,1)
trainx = trainx.reshape(trainx.shape[0],1,trainx.shape[1],trainx.shape[2]) # x = (153, 1, 9, 30)
feat_num = trainx.shape[2]
del train_frame
print("trainx.shape: ",trainx.shape)
print("trainy.shape: ",trainy.shape)

100%|██████████| 4032/4032 [10:04<00:00,  6.67it/s]


trainx.shape:  (837692, 1, 20, 30)
trainy.shape:  (837692, 1)


In [5]:
testx,testy = [],[]
test_target = pd.DataFrame()
for ticker in tqdm(test_frame['ticker'].drop_duplicates()):
    one_data = test_frame[test_frame['ticker'] == ticker]
    one_data = one_data.set_index(['timestamp','ticker'])
    array = np.array(one_data)
    one_data = one_data.reset_index()
    for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        testx.append(array[i:i+day,:-1].T)
        testy.append(array[i+day-1][-1])
        temp = pd.DataFrame(one_data.iloc[i+day-1,:]).T
        test_target = pd.concat([test_target,temp[['timestamp','ticker','target']]],axis=0) 
#         test_target = pd.concat([test_target,pd.DataFrame(one_data.iloc[i+day-1,-1])])
testx  , testy = np.array(testx) , np.array(testy).reshape(-1,1) # x = (153, 9, 30) , y = (153,1)
testx = testx.reshape(testx.shape[0],1,testx.shape[1],testx.shape[2]) # x = (153, 1, 9, 30)
del test_frame
print("testx.shape: ",testx.shape)
print("testy.shape: ",testy.shape)
test_target.reset_index(inplace = True,drop = True)
test_target

100%|██████████| 4093/4093 [05:53<00:00, 11.58it/s]


testx.shape:  (99072, 1, 20, 30)
testy.shape:  (99072, 1)


,timestamp,ticker,target
0,2021-05-18 09:30:00,002893,1.830067
1,2021-05-21 09:30:00,002893,-0.18859
2,2021-05-26 09:30:00,002893,0.846915
3,2021-05-31 09:30:00,002893,-0.333178
4,2021-06-03 09:30:00,002893,-1.997481
...,...,...,...
99067,2021-06-24 09:30:00,600179,-0.123318
99068,2021-06-24 09:30:00,600243,-0.74492
99069,2021-06-24 09:30:00,002426,-0.333088
99070,2021-06-25 09:30:00,000536,-0.405932


In [6]:
class Convolutional(object):
    def __init__(self,data,stride):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = np.array(data)
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2] # 9
        self.num , self.num_rev = self.generate_Num_and_ReversedNum(self.feat_num)
        self.conv_feat = len(self.num)
        self.step_list = self.generate_Step_List(self.data_length,self.stride)
        self.extracted_data = self.Extraction(self.data,self.feat_num,self.conv_feat,self.stride)
    def Extraction(self,data,feat_num,conv_feat,stride):
        print("------Start Extraction------")
        batch = nn.BatchNorm1d(conv_feat,affine=True)
        batch2 = nn.BatchNorm1d(feat_num,affine=True)
        conv1 = self.ts_cov4d(self.data,self.stride,self.num,self.num_rev,self.step_list)
        conv2 = self.ts_corr4d(self.data,self.stride,self.num,self.num_rev,self.step_list,conv1)
        conv2 = torch.tanh(conv2)
        bc1 = batch(conv1.to(torch.float))
        bc2 = batch(conv2.to(torch.float))
        conv3 = self.ts_stddev4d(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc3 = batch2(conv3)
        conv4 = self.ts_zscore(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc4 = batch2(conv4)
        conv5 = self.ts_return(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc5 = batch2(conv5)
        conv6 = self.ts_decaylinear(self.data,self.stride,self.feat_num,self.step_list).to(torch.float)
        bc6 = batch2(conv6)

        feat_cat = torch.cat([bc1,bc2,bc3,bc4,bc5,bc6],axis = 1) # 特征聚合
        shape = feat_cat.shape
        feat_cat = feat_cat.reshape(shape[0],1,shape[1],shape[2])
        print("Convolutional shape: ", feat_cat.shape)
        return feat_cat
    def generateC(self,l1):
        if len(l1) == 1:
            return []
        v = [[l1[0],i] for i in l1[1:]]
        l1 = l1[1:]
        return v+ self.generateC(l1)
    def generate_Num_and_ReversedNum(self,feat_nums):
        list1 = list(range(feat_nums))
        num = self.generateC(list1)
        num_rev = []
        for l in num:
            l1 = l.copy()
            l1.reverse()
            num_rev.append(l1)
        return num , num_rev
    def generate_Step_List(self,data_length,stride):
        # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
    
    
    """ Main Extraction"""
    def ts_cov4d(self,data,stride,num,num_rev,step_list):
        '''计算4维数据的协方差'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        #计算的过程中务必保持keepdims=True
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
            sub_data2 = data[:,:,num_rev,start:end]
            mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
            mean2 = sub_data2.mean(axis = 4,keepdims = True)
            spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
            spread2 = sub_data2 - mean2
            cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
            l.append(cov)
        corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,self.conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
        final = torch.from_numpy(corr)
        print("------Finished ts_cov4d----output shape: ",final.shape)
        return final
    def ts_corr4d(self,data,stride,num,num_rev,step_list,cov):
        '''计算4维数据的相关系数'''
        '''data:[N,C,H,W],,W:price length,N:batch size'''
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,num,start:end]
            sub_data2 = data[:,:,num_rev,start:end]
            std1 = sub_data1.std(axis = 4,keepdims = True)
            std2 = sub_data2.std(axis = 4,keepdims = True)
            std = (std1*std2).mean(axis = 3,keepdims = True)
            del std1 , std2 # 释放内存
            l.append(std)
        std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,self.conv_feat,len(step_list)-1)
        std[std == 0] = 1e-9
        fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
        final = cov / torch.from_numpy(std) * fct
        del fct , std
        print("------Finished ts_corr4d----output shape: ",final.shape)
        return final
    def ts_stddev4d(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            std1 = sub_data1.std(axis = 3,keepdims = True)
            l.append(std1)
            del std1
        std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        print("------Finished ts_stddev4d----output shape: ",torch.from_numpy(std).shape)
        return torch.from_numpy(std)
    def ts_zscore(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            mean = sub_data1.mean(axis = 3,keepdims = True)
            std = sub_data1.std(axis = 3,keepdims = True)
            std[std == 0] = 1e-9
            z_score = mean/std
            l.append(z_score)
        z_score = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
#         z_score[z_score >= 6] = 6
        print("------Finished ts_zscore----output shape: ",torch.from_numpy(z_score).shape)
        return torch.from_numpy(z_score)
    def ts_return(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        data[data == 0] = 1e-9
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            sub_data1 = data[:,:,:,start:end]
            ret = sub_data1[:,:,:,-1]/sub_data1[:,:,:,0] - 1
            l.append(ret)
        z_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        z_data[z_data > 1] = 1
        print("------Finished ts_return----output shape: ",torch.from_numpy(z_data).shape)
        return torch.from_numpy(z_data)
    def ts_decaylinear(self,data,stride,feat_num,step_list):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            time_spread = end - start
            weight = np.arange(1,time_spread+1)
            weight = weight/(weight.sum())
            sub_data1 = (data[:,:,:,start:end]*weight).mean(axis = 3,keepdims = True)
            l.append(sub_data1)
        decay_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list)-1)
        final = torch.from_numpy(decay_data)
        print("------Finished ts_decaylinear----output shape: ",final.shape)
        return final

In [7]:
class Pooling(object):
    def __init__(self,data,stride):
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        self.data = data.detach().numpy()
        self.stride = stride
        self.data_length = data.shape[3]
        self.feat_num = data.shape[2] # 9
        self.step_list = self.generate_Step_List(self.data_length,self.stride)
        self.extracted_data = self.Extraction(self.data,self.feat_num,self.stride)
    def Extraction(self,data,feat_num,stride):
        print("------Start Pooling------")
        # Pooling
        ts_max = self.ts_pool(feat_cat,self.stride,self.feat_num,self.step_list,method = 'max')
        ts_max = nn.BatchNorm1d(self.feat_num,affine = True)(ts_max)
        ts_min = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'min')
        ts_min = nn.BatchNorm1d(self.feat_num,affine = True)(ts_min)
        ts_mean = self.ts_pool(feat_cat ,self.stride,self.feat_num,self.step_list,method = 'mean')
        ts_mean = nn.BatchNorm1d(self.feat_num,affine = True)(ts_mean)
        data_pool = torch.cat([ts_max,ts_min,ts_mean],axis = 1)
        data_pool = data_pool.flatten(start_dim = 1)
        print("Pooling shape: ",data_pool.shape )
        return data_pool
    def generate_Step_List(self,data_length,stride):
        # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
        if data_length % stride == 0:
            step_list = list(range(0,data_length+stride,stride))
        elif data_length % stride<=5:
            mod = data_length % stride
            step_list = list(range(0,data_length-stride,stride))+[data_length]
        else:
            mod = data_length % stride
            step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
        return step_list
    def ts_pool(self,data,stride,feat_num,step_list,method):
        if type(data) == torch.Tensor:
            data = data.detach().numpy()
        if data.shape[-1] <= stride:
            step_list = [0,data.shape[-1]]
        if len(data.shape)!=4:
            raise Exception('Input data dimensions should be [N,C,H,W]')
        l = []
        for i in tqdm(range(len(step_list)-1)):
            start = step_list[i]
            end = step_list[i+1]
            if method == 'max':
                sub_data1 = data[:,:,:,start:end].max(axis = 3,keepdims = True)
            if method == 'min':
                sub_data1 = data[:,:,:,start:end].min(axis = 3,keepdims = True)
            if method == 'mean':
                sub_data1 = data[:,:,:,start:end].mean(axis = 3,keepdims = True)
            l.append(sub_data1)
        try:
            pool_data = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,feat_num,len(step_list) - 1)
        except:
            pool_data = np.squeeze(np.array(l)).reshape(-1,feat_num,len(step_list) - 1)
        return torch.from_numpy(pool_data)

In [ ]:
convolutional = Convolutional(trainx,10)
feat_cat = convolutional.extracted_data
pooling = Pooling(feat_cat,3)
trainx = pooling.extracted_data.detach().numpy()
print("trainx.shape : ",trainx.shape)
print("trainy.shape : ",trainy.shape)

  0%|          | 0/3 [00:00<?, ?it/s]

------Start Extraction------


In [ ]:
convolutional = Convolutional(testx,10)
feat_cat = convolutional.extracted_data
pooling = Pooling(feat_cat,3)
testx = pooling.extracted_data.detach().numpy()
print("testx.shape : ",testx.shape)
print("testy.shape : ",testy.shape)

In [ ]:
import torch
from torch import nn
class AlphaNet(nn.Module):
    def __init__(self,factor_num,fully_connect_layer_neural):
        # super 父类，调用父类的构造，这一步必须有
        # 第一个参数为定义类的名称，第二个为self
        super(AlphaNet,self).__init__()
        self.fc1_neuron = (factor_num * (factor_num -1) + 4 * factor_num)* 3
        self.fc2_neuron = fully_connect_layer_neural
        self.fc1 = torch.nn.Linear(self.fc1_neuron,self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron,1)
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred

In [ ]:
trainx , trainy  , testx , testy = torch.from_numpy(trainx) , torch.from_numpy(trainy) , torch.from_numpy(testx) , torch.from_numpy(testy)
print('trainx size: ', trainx.size())
print('trainy size: ', trainy.size())
print('testx size: ', testx.size())
print('testy size: ', testy.size())

In [ ]:
import torch.utils.data as Data
train_dataset = Data.TensorDataset(trainx, trainy)
test_dataset = Data.TensorDataset(testx, testy)
batch_size = 128
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = Data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

In [ ]:
import torch.optim as optim 
from tqdm import tqdm
from torch.autograd import Variable

alphanet = AlphaNet(feat_num,30)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.000001
loss_function = nn.MSELoss()
optimizer = optim.RMSprop(alphanet.parameters(), lr=LR, alpha=0.9)
epoch_num = 30


for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _,(data, label) in enumerate(train_loader):
        data = Variable(data).float()
        pred = alphanet(data)
        label = Variable(label).float()
#         label = label.unsqueeze(1)
        loss = loss_function(pred, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch+1, ' loss: ', total_loss)

In [ ]:
pred_list = []
label_list = []

for _, (data, label) in enumerate(test_loader):
    data = Variable(data).float()
    pred = alphanet(data)
    pred_list.extend(pred.tolist())
    label_list.extend(label.tolist())

In [ ]:
for i in range(1000):  
    print("pred: ", pred_list[i][0])
    print("true: ",label_list[i][0])
    print("---------------------")

In [ ]:
final = pd.concat([test_target,pd.DataFrame(pred_list)],axis=1)
final = final[['timestamp','ticker',0]]
alpha_name = 'AlphaNet'
final.rename(columns={ 0 : alpha_name ,'ticker': 'symbol'}, inplace=True)
final = final.reindex(columns=['symbol', 'timestamp', alpha_name])

In [ ]:
final.set_index(['symbol','timestamp']).to_csv('/home/wuwenjun/Alpha_Factor/Alpha_%s' % time_end)

In [ ]:
%ls /home/wuwenjun/Alpha_Factor/